In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [29]:
#做一个遍历的字典
sentences = ['i like cat', 'i love coffee', 'i hate milk']
sentences_list = " ".join(sentences).split() # ['i', 'like', 'cat', 'i', 'love'. 'coffee',...]
vocab = list(set(sentences_list)) # 去重复
word2idx = {w:i for i, w in enumerate(vocab)}
idx2word = {i:w for i, w in enumerate(vocab)}

V = len(vocab)

In [30]:
def make_data(sentences):
    input_data = []
    target_data = []#预测下一个单词
    #句子的向量化tmp
    for sen in sentences:
        sen = sen.split()
        input_tmp = [word2idx[w] for w in sen[:-1] ]#前n-1个单词
        target_tmp = word2idx[sen[-1]]#预测的最后一个单词

        input_data.append(input_tmp)
        target_data.append(target_tmp) 
    return input_data, target_data


In [31]:
input_data,target_data =  make_data(sentences)

input_data = torch.LongTensor(input_data)
target_data =  torch.LongTensor(target_data)
dataset = Data.TensorDataset(input_data, target_data)
loader = Data.DataLoader(dataset, 2, True)#数据源，batch_size, shuffle



In [32]:
#parameters
m = 2
n_step = 2 #输入的单词数目
n_hidden = 2 #隐藏层的神经元个数

In [33]:
class NNLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.C = nn.Embedding(V, m)
        self.H = nn.Parameter(torch.randn(n_step * m, n_hidden).type(dtype))  #dtype = torch.FloatTensor
        self.d = nn.Parameter(torch.randn(n_hidden).type(dtype))
        self.b = nn.Parameter(torch.randn(V).type(dtype))
        self.W = nn.Parameter(torch.randn(n_step * m, V).type(dtype))
        self.U = nn.Parameter(torch.randn(n_hidden, V).type(dtype))
    def forward(self, X):
        '''
        X: [batch_size, n_step]

        '''
        X = self.C(X)
        X = X.view(-1, n_step * m) # [batch_size, n_step * m] 三维压二维
        hidden_out = torch.tanh(X.mm(self.H)+self.d) # [batch_size, n_hidden]
        output = self.b + X.mm(self.W) + hidden_out.mm(self.U) # [batch_size, V]
        return output

model = NNLM()
optim = optim.Adam(model.parameters(),lr=1e-3) #定义优化器
criterion = nn.CrossEntropyLoss()   #定义损失函数

        

In [35]:
for epoch in range(5000):
    for i, (batch_x, batch_y) in enumerate(loader):
        pred = model(batch_x)
        loss = criterion(pred, batch_y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        if (epoch+1) % 1000 == 0:
            print("epoch: {}, loss: {:.9f}".format(epoch+1, loss.item()))

epoch: 1000, loss: 0.000041961
epoch: 1000, loss: 0.000020265
epoch: 2000, loss: 0.000013709
epoch: 2000, loss: 0.000007629
epoch: 3000, loss: 0.000003695
epoch: 3000, loss: 0.000004768
epoch: 4000, loss: 0.000001371
epoch: 4000, loss: 0.000001669
epoch: 5000, loss: 0.000000596
epoch: 5000, loss: 0.000000477


In [40]:
#Pred
predict = model(input_data).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [idx2word[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['cat', 'coffee', 'milk']


In [2]:
import torch

# 检查是否有可用的 GPU
if torch.cuda.is_available():
    print("CUDA is available. GPU will be used.")
    # 获取当前 GPU 设备
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. CPU will be used.")
    device = torch.device("cpu")

# 创建一个随机张量并将其移动到 GPU
tensor = torch.randn(10000, 10000).to(device)

# 执行一些计算
result = tensor @ tensor.T

print("Calculation completed.")
print("Embedding calculation completed.")

CUDA is not available. CPU will be used.
Calculation completed.
Embedding calculation completed.
